In [1]:
import pandas as pd
import numpy as np

num_data_points = 100000


start_date = pd.to_datetime('2023-06-09 08:00:00')

timestamps = pd.date_range(start_date, periods=num_data_points, freq='1min')
sensory_info = np.random.uniform(0.5, 2.5, num_data_points)
cycles = np.random.randint(1, 20, num_data_points)
temperature = np.random.uniform(45.0, 46.0, num_data_points)
torque = np.random.uniform(23.0, 25.0, num_data_points)
pressure = np.random.uniform(2.0, 3.0, num_data_points)
failure = np.random.randint(0, 2, num_data_points)

data = pd.DataFrame({
    'Timestamp': timestamps,
    'Sensory Information': sensory_info,
    'Cycle': cycles,
    'Temperature (Celsius)': temperature,
    'Torque (Nm)': torque,
    'Pressure (bar)': pressure,
    'Failure': failure
})

data.to_csv('sensory_data.csv', index=False)

In [11]:
data

,Timestamp,Sensory Information,Cycle,Temperature (Celsius),Torque (Nm),Pressure (bar),Failure
0,2023-06-09 08:00:00,0.791983,19,45.707800,23.358061,2.613868,1
1,2023-06-09 08:01:00,1.734442,17,45.472720,24.421239,2.508511,0
2,2023-06-09 08:02:00,1.815415,19,45.892473,23.384211,2.458794,0
3,2023-06-09 08:03:00,1.817874,5,45.004059,23.384611,2.846745,0
4,2023-06-09 08:04:00,1.860960,2,45.456774,23.949539,2.945963,1
...,...,...,...,...,...,...,...
99995,2023-08-17 18:35:00,1.717324,4,45.339491,23.925345,2.638350,1
99996,2023-08-17 18:36:00,1.378313,18,45.548971,24.225442,2.631890,1
99997,2023-08-17 18:37:00,0.742788,16,45.251671,24.320942,2.912132,0
99998,2023-08-17 18:38:00,2.176215,3,45.744340,24.249474,2.559665,0


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense


df = pd.read_csv('sensory_data.csv')


scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['Sensory Information', 'Cycle']])


train_size = int(len(df) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]


def create_sequences(data, sequence_length):
    X = []
    y = []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length][-1])
    return np.array(X), np.array(y)


sequence_length = 3


X_train, y_train = create_sequences(train_data, sequence_length)


X_test, y_test = create_sequences(test_data, sequence_length)


X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 2))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 2))


model = Sequential()
model.add(LSTM(units=64, input_shape=(sequence_length, 2)))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy')


model.fit(X_train, y_train, epochs=10, batch_size=32)


loss = model.evaluate(X_test, y_test)
print('Testing loss:', loss)


predictions = model.predict(X_test)

Epoch 1/10
2500/2500 [==============================] - 14s 5ms/step - loss: 0.6933
Epoch 2/10
2500/2500 [==============================] - 11s 5ms/step - loss: 0.6932
Epoch 3/10
2500/2500 [==============================] - 11s 4ms/step - loss: 0.6932
Epoch 4/10
2500/2500 [==============================] - 11s 4ms/step - loss: 0.6932: 0s - loss: 0.6 - ETA: 0s - loss: 0.693
Epoch 5/10
2500/2500 [==============================] - 11s 4ms/step - loss: 0.6932
Epoch 6/10
2500/2500 [==============================] - 11s 4ms/step - loss: 0.6932
Epoch 7/10
2500/2500 [==============================] - 11s 4ms/step - loss: 0.6932
Epoch 8/10
2500/2500 [==============================] - 11s 4ms/step - loss: 0.6932
Epoch 9/10
2500/2500 [==============================] - 11s 4ms/step - loss: 0.6932
Epoch 10/10
625/625 [==============================] - 2s 3ms/step - loss: 0.6932
Testing loss: 0.6931516528129578


In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


data = pd.read_csv('sensory_data.csv')

X = data[['Sensory Information', 'Cycle', 'Temperature (Celsius)', 'Torque (Nm)', 'Pressure (bar)']]
y = data['Failure']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.4996
